In [58]:
# -*- coding:utf-8 -*-
import sys
import json
import numpy as np
import paddle.v2 as paddle

In [2]:
paddle.init(use_gpu=False, trainer_count=1)

In [3]:
input_data = paddle.layer.data(name='data', type=paddle.data_type.dense_vector_sequence(2048))

In [4]:
def rnn(input_seq):
    
    bidirectional_lstm_1 = paddle.networks.bidirectional_lstm(
        input=input_seq,
        size=512,
        return_seq=True
    )
    
    bidirectional_lstm_2 = paddle.networks.bidirectional_lstm(
        input=bidirectional_lstm_1,
        size=512,
        return_seq=True
    )
    
    fc_1 = paddle.layer.fc(
        input=bidirectional_lstm_2,
        size=512,
        act=paddle.activation.Relu()
    )
    
    fc_2 = paddle.layer.fc(
        input=fc_1,
        size=100,
        act=paddle.activation.Relu()
    )
    
    output = paddle.layer.fc(
        input=fc_2,
        size=2,
        act=paddle.activation.Softmax()
    )
    return output
    

In [5]:
# Fake Data
def create_reader(length):
    
    def reader():
            
        video_length = np.random.randint(3000,4000)

        data = np.random.rand(video_length, 2048)
        labels = np.random.randint(0,2,video_length)

        for i in xrange(video_length/length):
            yield data[i*length: (i+1)*length],labels[i*length: (i+1)*length]
    return reader

In [6]:
def event_handler(event):
        if isinstance(event, paddle.event.EndIteration):
            if event.batch_id % 20 == 0:
                print "\nPass %d, Batch %d, Cost %f, %s" % (
                    event.pass_id, event.batch_id, event.cost, event.metrics)
            else:
                sys.stdout.write('.')
                sys.stdout.flush()
        if isinstance(event, paddle.event.EndPass):
            with open('./params_pass_%d.tar' % event.pass_id, 'w') as f:
                trainer.save_parameter_to_tar(f)

In [7]:
is_train = True

In [8]:
output = rnn(input_data)

if is_train:
    input_label = paddle.layer.data(name='label', type=paddle.data_type.integer_value_sequence(2))
    cost = paddle.layer.cross_entropy_cost(input=output, label=input_label)
    train_reader = create_reader(10)
    train_reader = paddle.batch(reader=train_reader,batch_size=32)
    feeding = {'data':0, 'label':1}
    adam_optimizer = paddle.optimizer.Adam(learning_rate=2e-4,
        regularization=paddle.optimizer.L2Regularization(rate=8e-4),
        model_average=paddle.optimizer.ModelAverage(average_window=0.5))
    
    parameters = paddle.parameters.create(cost)
    trainer = paddle.trainer.SGD(cost=cost,parameters=parameters,update_equation=adam_optimizer)

    trainer.train(reader=train_reader,event_handler=event_handler,feeding=feeding,num_passes=4)


Pass 0, Batch 0, Cost 6.924755, {}
...........
Pass 1, Batch 0, Cost 7.097571, {}
..........
Pass 2, Batch 0, Cost 6.929973, {}
...........
Pass 3, Batch 0, Cost 6.969117, {}
............

In [9]:
def generate_inference_data():
    
    video_length = np.random.randint(3000,4000)

    data = np.random.rand(video_length, 2048)
    
    return data

In [12]:
def inference(x):
    with open('params_pass_3.tar', 'r') as f:
        parameters = paddle.parameters.Parameters.from_tar(f)
    probs = paddle.infer(output_layer=output, parameters=parameters, input=[[x,]], feeding={'data': 0})
    
    return probs
    

In [13]:
infer_x = generate_inference_data()
result = inference(infer_x)

In [14]:
predict = np.argmax(result, axis=1)

In [55]:
def generate_result(prediction, probs):
    
    # result: [((start, end),prob),....]
    result = []
    
    current_start = -1
    current_end = 0
    # 间隔空白
    blank_threshhold = 10
    # 最小长度
    min_lengths = 5
    
    state = 0
    # state = 0 未找到start
    # state = 1 找到start，未找到end, 当前值为1
    # state = 2 找到start，未找到end, 当前值为0
    # state = 3 中间隔了很多空白

    for i in xrange(prediction.shape[0]):
        if state == 0 and prediction[i] == 1:
            
            current_start = i
            state = 1
            continue
            
        if state == 1 and prediction[i] == 1:
            
            current_end = i
            continue
        
        if state == 1 and prediction[i] == 0:
            
            state = 2
            continue
        
        if state == 2 and prediction[i] == 1:
            
            distance = i - current_end
            if distance > blank_threshhold:
                
                length = current_end - current_start + 1
                prob = sum(probs[current_start: current_end+1])/length
                current_start = -1
                current_end = -1
                state = 0
                if length <= min_lengths:
                    current_start = -1
                    current_end = -1
                    continue
                result.append(([current_start, current_end],prob))
                current_start = -1
                current_end = -1
                continue
            else:
                
                if distance < blank_threshhold/2:
                    current_end = i
                    state = 1
                    continue
                else:
                    state = 3
                    continue
                    
        if state == 3 and prediction[i] == 1:
            current_end = i
            state = 1
            continue
        
        if state == 3 and prediction[i] == 0:
            
            length = current_end - current_start + 1
            prob = sum(probs[current_start: current_end+1])/length

            state = 0
            if length <= min_lengths:
                current_start = -1
                current_end = -1
                continue
            result.append(([current_start, current_end],prob))
            current_start = -1
            current_end = -1
            continue
        
    return result


In [18]:
def fake_prediction(size):
    x0 = np.random.random(size=[size, 1])
    x1 = 1-x0
    s = np.stack([x0, x1], axis=1)
    s = s.reshape([size, 2])
    return s

In [30]:
fake_result = fake_prediction(1000)
fake_index = np.argmax(fake_result, axis=1)
fake_prob = fake_result[:,1]

In [31]:
fake_result[1:10]

array([[ 0.8981382 ,  0.1018618 ],
       [ 0.25008401,  0.74991599],
       [ 0.67514029,  0.32485971],
       [ 0.94494107,  0.05505893],
       [ 0.20734265,  0.79265735],
       [ 0.27510173,  0.72489827],
       [ 0.12586007,  0.87413993],
       [ 0.37334489,  0.62665511],
       [ 0.79121609,  0.20878391]])

0.53437094615038128

In [56]:
result = generate_result(fake_index, fake_prob)

In [57]:
result

[((0, 288), 0.53252191173463603),
 ((323, 410), 0.58116175497752287),
 ((428, 513), 0.53141422877930755),
 ((522, 566), 0.5609105026149096),
 ((577, 583), 0.68735125710388523),
 ((624, 719), 0.5411929583538061),
 ((726, 934), 0.49511491584896694)]

In [59]:
def generate_result_file(data, filepath, version='VERSION 1.0'):
    result = dict()
    result['version'] = version
    
    results = dict()
    
    for key in data.keys():
        
        tmp_result = []
        for seg in data[key]:
            tmp_dict = dict()
            tmp_dict['score'] = seg[1]
            tmp_dict['segment'] = seg[0]
        tmp_result.append(tmp_dict)
        results[key] = tmp_result
    
    result['results'] = results
    
    r = json.dumps(results)
    with open(filepath, 'w+') as f:
        f.write(r)
    print("generate finished")

In [60]:
result = {"vdwaf": result}

In [61]:
generate_result_file(result, 'ss.json')

generate finished
